In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [4]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

train.drop(['is_attributed', 'day', 'timestamp'], axis=1, inplace = True)
test.drop(['day', 'timestamp'], axis=1, inplace = True)

In [5]:
test = test[list(train.columns)]
df_all = pd.concat([train,test])

train_length = len(train)
test_length = len(test)

import gc
del train
del test
gc.collect()

label = np.concatenate([np.zeros(train_length), np.ones(test_length)]) 
df_all['label'] = label

import sys
print(sys.getsizeof(df_all) / 1024**3)


15.176412411034107


# prepare train, val

In [8]:
from sklearn.model_selection import train_test_split
trainset, valset = train_test_split(df_all.sample(frac=1, random_state=233),test_size=0.25, random_state=19)

scale_pos_weight = len(trainset[trainset['label'] == 0]) / len(trainset) * 100
print('train negative rate: {}'.format(scale_pos_weight))
print('val negative rate: {}'.format(len(valset[valset['label'] == 0]) / len(valset) * 100))

train negative rate: 90.77647111928854
val negative rate: 90.77124570361202


In [9]:
categorical_col = ['ip', 'app', 'device', 'os', 'channel', 'hour', 'minute', 'second']
feature_cols = ['ip', 'app', 'device', 'os', 'channel', 'hour', 'minute', 'second']

# Train LGB

In [12]:
target = 'label'
y_train = trainset[target].values
y_val = valset[target].values

lgb_train = lgb.Dataset(trainset[feature_cols], y_train, categorical_feature = categorical_col)
lgb_val = lgb.Dataset(valset[feature_cols], y_val, categorical_feature = categorical_col)

zeros = len(y_train[y_train == 0])
# scale_pos_weight = len(y_train[y_train == 0]) / len(y_train[y_train == 1])
import gc
gc.collect()

0

In [13]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 300,
        'learning_rate': 0.1,
        'num_leaves': 9,
        'num_threads': 20, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 20,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':30,
        'bagging_fraction': 0.8, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'scale_pos_weight': scale_pos_weight,
        'metric' : 'auc'
    }

model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 30 rounds.
[10]	valid_0's auc: 0.883313
[20]	valid_0's auc: 0.88595
[30]	valid_0's auc: 0.888314
[40]	valid_0's auc: 0.890169
[50]	valid_0's auc: 0.892969
[60]	valid_0's auc: 0.893853
[70]	valid_0's auc: 0.894424
[80]	valid_0's auc: 0.894767
[90]	valid_0's auc: 0.895466
[100]	valid_0's auc: 0.895578
[110]	valid_0's auc: 0.8958
[120]	valid_0's auc: 0.896293
[130]	valid_0's auc: 0.896571
[140]	valid_0's auc: 0.896635
[150]	valid_0's auc: 0.896687
[160]	valid_0's auc: 0.89708
[170]	valid_0's auc: 0.897192
[180]	valid_0's auc: 0.897453
[190]	valid_0's auc: 0.897546
[200]	valid_0's auc: 0.897936
[210]	valid_0's auc: 0.898028
[220]	valid_0's auc: 0.898227
[230]	valid_0's auc: 0.898383
[240]	valid_0's auc: 0.898407
[250]	valid_0's auc: 0.898616
[260]	valid_0's auc: 0.898802
[270]	valid_0's auc: 0.899011
[280]	valid_0's auc: 0.899162
[290]	valid_0's auc: 0.899318
[300]	valid_0's auc: 0.899475
[310]	valid_0's auc: 0.899621
[320]	valid_0's auc: 

In [15]:
from sklearn.metrics import roc_auc_score
pred = model.predict(df_all[feature_cols])
df_all['pred'] = pred
print(roc_auc_score(df_all[target].values,pred))

0.90168976534


In [16]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
print(importance)

ip         2725
channel     415
app         341
hour        237
os          172
minute       60
device       32
second       18
dtype: int64


In [17]:
train = df_all.iloc[:train_length]
train['pred'].to_csv('/home/kai/data/kaggle/talkingdata/wl/data/trainset/train_prediction_0405_auc0.9016.csv')
train_final = train.sort_values(['pred'], ascending = False)

# sss = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/trainset/train_prediction.csv',header = None)
# sss.columns = ['index', 'preds']

train_final.head(20)

,ip,app,device,os,channel,hour,minute,second,label,pred
165495962,55887,21,2,36,232,10,9,49,0.0,0.999871
180818041,55887,9,1,23,127,14,40,44,0.0,0.999851
181507514,55887,9,59,17,107,14,53,10,0.0,0.999844
178383503,55887,21,1,6,232,13,59,8,0.0,0.999842
180029704,55887,22,1,27,116,14,26,55,0.0,0.999809
179977504,55887,9,1,6,107,14,26,1,0.0,0.999792
179972765,55887,9,1,6,107,14,25,57,0.0,0.999792
180046781,55887,9,1,27,107,14,27,12,0.0,0.999787
165502973,55887,9,1,6,107,10,9,57,0.0,0.999781
168068086,55887,9,1,13,107,10,56,33,0.0,0.999777


In [26]:
def get_count(df,col,segment):
    step = 1/ segment
    count = {}
    start = 1
    for i in range(segment):
        upper = start
        lower = start - step
        start -= step
        count[i] = len(df[(df[col] <= upper) & (df[col] > lower) ])
    return count

count = get_count(train_final, 'pred', 10)

In [27]:
count

{0: 46745331,
 1: 644715,
 2: 671189,
 3: 428552,
 4: 52710,
 5: 68268,
 6: 143594,
 7: 291485,
 8: 7592907,
 9: 128265139}

In [30]:
train_final['index'] = train_final.index.values

# strategy 1: Get All until hit threshold

In [20]:
thresh = 55 * 1000000
index = list(train_final.index.values[:thresh])
val_length = 10 * 1000000

## 1. top 10 m as val

In [22]:
val_index = index[:val_length]
train_index = index[val_length:]

file_val = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/val_index_all_top.npy'
file_train = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/train_index_all_top.npy'

np.save(file_val, val_index)
np.save(file_train, train_index)

## 2. shuffle as val

In [25]:
from sklearn.model_selection import train_test_split
ratio = val_length / thresh
train_index, val_index = train_test_split(index,test_size=ratio, random_state=19)

file_val = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/val_index_all_shuffle.npy'
file_train = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/train_index_all_shuffle.npy'

np.save(file_val, val_index)
np.save(file_train, train_index)

## strategy 2: Get 0.75 until hit threshold

In [32]:
def get_count(df,col,segment):
    step = 1/ segment
    count = {}
    start = 1
    for i in range(segment):
        upper = start
        lower = start - step
        start -= step
        count[i] = len(df[(df[col] <= upper) & (df[col] > lower) ])
    return count
        
    
def get_index_list(df,col,segment, total, ratio=0.75, sort = False ):
    start = 1
    step = 1 / segment
    seed = list(range(1, segment + 1))
    dict_count = get_count(df, col, segment)
    print('finish dict count')
    index_list = []
    remain = total
    for i in range(segment):
        print(i)
        upper = start
        lower = start - step
        start -= step
        tmp = df[(df[col] <= upper) & (df[col] > lower) ]
        tmp_index = tmp['index'].values
        ran = np.random.RandomState(seed = seed[i])
        ran.shuffle(tmp_index)
        ### get index
        num_index = int(ratio * dict_count[i])
        if num_index > remain:
            num_index = remain
            index_list.extend(list(tmp_index[:num_index]))
            print(len(index_list))
            break
#         index_list.extend(list(tmp_index[:num_index]))
        index_list.extend(tmp_index[:num_index].tolist())
        remain -= num_index
        print(len(index_list))
    if sort:
        index_list.sort()
    return index_list

segment = 10
ratio = 0.85
index_partial = get_index_list(train_final, 'pred', 10, thresh, ratio)

import sys
print(sys.getsizeof(index_partial) / 1024 **3)

finish dict count
0
39733531
1
40281538
2
40852048
3
41216317
4
41261120
5
41319147
6
41441201
7
41688963
8
48142933
9
55000000
0.4610047787427902


# 1. top 10 m as val 

In [34]:
val_index = index_partial[:val_length]
train_index = index_partial[val_length:]

file_val = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/val_index_partial_0.85_top.npy'
file_train = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/train_index_partial_0.85_top.npy'

np.save(file_val, val_index)
np.save(file_train, train_index)

## 2. shuffle as val

In [35]:
from sklearn.model_selection import train_test_split
ratio = val_length / thresh
train_index, val_index = train_test_split(index_partial,test_size=ratio, random_state=19)

file_val = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/val_index_partial_0.85_shuffle.npy'
file_train = '/home/kai/data/kaggle/talkingdata/wl/data/trainset/train_index_partial_0.85_shuffle.npy'

np.save(file_val, val_index)
np.save(file_train, train_index)